<a href="https://colab.research.google.com/github/Thoufiq-Developer/REAL-OR-FAKE-NEWS-PREDICTION/blob/main/fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [119]:
b=nltk.download('stopwords')
a=stopwords.words('english')


print(a)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


DATA PREPROCESSING


In [120]:
data=pd.read_csv('/content/train (1).csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [121]:
data.shape

(20800, 5)

In [122]:
data.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [123]:
data.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [124]:
data=data.fillna('')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [125]:
data.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [126]:
data['content']=data['author']+'  '+ data['title']
print(data['content'])

0        Darrell Lucus  House Dem Aide: We Didn’t Even ...
1        Daniel J. Flynn  FLYNN: Hillary Clinton, Big W...
2        Consortiumnews.com  Why the Truth Might Get Yo...
3        Jessica Purkiss  15 Civilians Killed In Single...
4        Howard Portnoy  Iranian woman jailed for ficti...
                               ...                        
20795    Jerome Hudson  Rapper T.I.: Trump a ’Poster Ch...
20796    Benjamin Hoffman  N.F.L. Playoffs: Schedule, M...
20797    Michael J. de la Merced and Rachel Abrams  Mac...
20798    Alex Ansary  NATO, Russia To Hold Parallel Exe...
20799             David Swanson  What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [127]:
set_a=data.drop(columns='label',axis=1)
set_b=data['label']

In [128]:
set_a.head()


,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even ...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big W..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get Yo...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for ficti...


In [129]:
set_b.head()

,label
0,1
1,0
2,1
3,1
4,1


In [130]:
print(set_a)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [131]:
#stemming
port_stemmer=PorterStemmer()

In [132]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [133]:
stemming('content')

'content'

In [134]:
print(data['content'])

0        Darrell Lucus  House Dem Aide: We Didn’t Even ...
1        Daniel J. Flynn  FLYNN: Hillary Clinton, Big W...
2        Consortiumnews.com  Why the Truth Might Get Yo...
3        Jessica Purkiss  15 Civilians Killed In Single...
4        Howard Portnoy  Iranian woman jailed for ficti...
                               ...                        
20795    Jerome Hudson  Rapper T.I.: Trump a ’Poster Ch...
20796    Benjamin Hoffman  N.F.L. Playoffs: Schedule, M...
20797    Michael J. de la Merced and Rachel Abrams  Mac...
20798    Alex Ansary  NATO, Russia To Hold Parallel Exe...
20799             David Swanson  What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [135]:
new_set=data['content'].values
new_set_2=data['label'].values
print(new_set)

['Darrell Lucus  House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'
 'Daniel J. Flynn  FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'
 'Consortiumnews.com  Why the Truth Might Get You Fired' ...
 'Michael J. de la Merced and Rachel Abrams  Macy’s Is Said to Receive Takeover Approach by Hudson’s Bay - The New York Times'
 'Alex Ansary  NATO, Russia To Hold Parallel Exercises In Balkans'
 'David Swanson  What Keeps the F-35 Alive']


In [136]:
print(new_set_2)

[1 0 1 ... 0 1 1]


In [137]:
new_set_2.shape

(20800,)

In [138]:
vectorizer=TfidfVectorizer()
vectorizer.fit(new_set)
new_set=vectorizer.transform(new_set)


In [139]:
print(new_set)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 273725 stored elements and shape (20800, 25173)>
  Coords	Values
  (0, 904)	0.26354004814013343
  (0, 3862)	0.30579573877221844
  (0, 4507)	0.20531415441295317
  (0, 5508)	0.2993429551929777
  (0, 5800)	0.2502787762405247
  (0, 6145)	0.24677171892553343
  (0, 7574)	0.23047267305353566
  (0, 10387)	0.1844880289323935
  (0, 11307)	0.1532265401605094
  (0, 11409)	0.20615188166061463
  (0, 12528)	0.24883399099107747
  (0, 12902)	0.3024224900242886
  (0, 19171)	0.22537992364975484
  (0, 22289)	0.3484071341454308
  (0, 22649)	0.26575278886038384
  (0, 23355)	0.18006497451107856
  (1, 2544)	0.28998438336643234
  (1, 3075)	0.1531053111853744
  (1, 3509)	0.3775183944330703
  (1, 4298)	0.1902428965987476
  (1, 5469)	0.26240126155666194
  (1, 8420)	0.7045992054867244
  (1, 10134)	0.18787145765749735
  (1, 15149)	0.1586226371149596
  (1, 23748)	0.29662102960192643
  :	:
  (20797, 17253)	0.2367895682882451
  (20797, 17550)	0.302371441532

In [153]:
#splitting dataset to training and test data

X_train,X_test,Y_train,Y_test=train_test_split(new_set,new_set_2,test_size=0.2,stratify=new_set_2,random_state=2)

In [154]:
print(new_set.shape,new_set_2.shape,X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(20800, 25173) (20800,) (16640, 25173) (4160, 25173) (16640,) (4160,)


In [156]:
model=LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

In [157]:
#accuracy score

accuracy=accuracy_score(model.predict(X_train),Y_train)
print(accuracy)

0.9870192307692308


In [159]:
accuracy=accuracy_score(model.predict(X_test),Y_test)
print(accuracy)

0.9788461538461538


In [179]:
#MAKING A PREDICTIVE SYSTEM :

New_news=X_test[1220]
prediction=model.predict(New_news)
print(prediction)

if prediction==0:
  print("Real News")
else:
  print('Fake News')

[0]
Real News


Verification


In [180]:
print(Y_test[1220])

0


In [181]:
Y_test.shape

(4160,)